In [1]:
from __future__ import print_function 
from bs4 import BeautifulSoup
from BeautifulSoup import SoupStrainer as sopstrain
import subprocess
import os
import re
import numpy as np
import string
import operator

In [13]:
knowledgGraph = "cnet" # conceptnet5AssocMod, gov, conceptnet5AssocMi, conceptnet5AssocHdl
#knowledgGraph_ = ''.join([k.capitalize() if i>0 else k for i, k in enumerate(knowledgGraph)])
collection = "trec7n8"
method = "assoc" # hal, mi, assoc, assocMi, assocHal
projectDir = "/home/fj9124/projects/ir/seq_kb_ir/" 
indexDir = "/scratch/index/indri_5_7/"
colMethodConfigsDir = os.path.join(projectDir, "configs", collection, method)
cfgInFileName = os.path.join(projectDir, "configs", collection, "query" + collection.capitalize()) 
print("cfg infile Name =", cfgInFileName)
cfgOutFileName=os.path.join(colMethodConfigsDir, knowledgGraph, "indriRunQuery.cfg") 
print("cfg outfile Name =", cfgOutFileName)
colIndexDir = os.path.join(indexDir, collection) 
print("collection index Dir =", colIndexDir)
if knowledgGraph in {"cnet"}:
    knowledgGraphIndexDir = os.path.join(indexDir, collection)   
else:
    knowledgGraphIndexDir = os.path.join(indexDir, knowledgGraph)   
print("knowledgGraph index Dir =", knowledgGraphIndexDir)
graphsDir = os.path.join(projectDir, "graphs")
#methodGraphsDir = os.path.join(graphsDir, method)
if method == "hal":
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph + ".txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"mi", "assoc"}:
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph, "graph.txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocMi"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "mi", collection, "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocHal"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "hal", "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
qrelsDir = os.path.join(projectDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
colQrelsFileName = os.path.join(colQrelsDir, "qrels.csv")
print("col Qrels File Name =", colQrelsFileName)
runsFileName = os.path.join(projectDir, "runs", method, collection, knowledgGraph, "indriRunQuery.runs") 
print("runsFileName =", runsFileName)
evalsFileName = os.path.join(projectDir, "evals", method, collection, knowledgGraph, "indriRunQuery.evals") 
print("evalsFileName =", evalsFileName)
#expansionCount = 25

cfg infile Name = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/queryTrec7n8
cfg outfile Name = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/assoc/cnet/indriRunQuery.cfg
collection index Dir = /scratch/index/indri_5_7/trec7n8
knowledgGraph index Dir = /scratch/index/indri_5_7/trec7n8
methodGraphsFileName = ['/home/fj9124/projects/ir/seq_kb_ir/graphs/assoc/cnet/graph.txt']
col Qrels File Name = /home/fj9124/projects/ir/seq_kb_ir/qrels/trec7n8/qrels.csv
runsFileName = /home/fj9124/projects/ir/seq_kb_ir/runs/assoc/trec7n8/cnet/indriRunQuery.runs
evalsFileName = /home/fj9124/projects/ir/seq_kb_ir/evals/assoc/trec7n8/cnet/indriRunQuery.evals


In [14]:
def splitStemText(text):
    text = re.sub('/|-|\"|_',' ', text) # replace - and slash with space
    words = text.split()
    stemmedWords = []
    for w in words:
        w = re.sub('\(|\)|\'s|,','', w) # remove paranthesis, apostrophe s, comma
        w = re.sub('\'','', w) # remove apostrophe
        cmd = "dumpindex " + knowledgGraphIndexDir + " t " + w + " | head -n1"
        #print("cmd =", cmd)
        stemmedWords.append(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().split()[1])
    return (stemmedWords)

print ("testing:", splitStemText("going reading"))

testing: ['go', 'read']


In [29]:
rWords1 = dict()
rWords = dict()
if method != "dir":
    print(methodGraphsFileName[0])
    print("reading graphs...")
    with open(methodGraphsFileName[0], 'r') as graphFile:
        oWord = ""
        for i, row in enumerate(graphFile):
            columns = row.split('\t')
            if(len(columns)==1):
                oWord = columns[0].strip()
            else:
                rWord = columns[0].strip()
                meas = np.float(columns[1].strip())
                if oWord in rWords:
                    rWords[oWord] += [{rWord:meas}]
#                        rWords1[oWord][rWord] = meas
                else:
                    rWords[oWord] = [{rWord:meas}]
                    #rWords1[oWord] = {rWord:meas}
print("size of rWords =", len(rWords))

/home/fj9124/projects/ir/seq_kb_ir/graphs/assoc/cnet/graph.txt
reading graphs...
size of rWords = 212531


In [30]:
if(len(methodGraphsFileName)>1):
    print(methodGraphsFileName[1])
    
    if method != "dir":
        print("reading graphs...")
        with open(methodGraphsFileName[1], 'r') as graphFile:
            oWord = ""
            for i, row in enumerate(graphFile):
                columns = row.split('\t')
                if(len(columns)==1):
                    oWord = columns[0].strip()
                else:
                    rWord = columns[0].strip()
                    meas = np.float(columns[1].strip())
                    if oWord in rWords1:
#                        rWords1[oWord] += [{rWord:meas}]
                        rWords1[oWord][rWord] = meas
                    else:
#                        rWords1[oWord] = [{rWord:meas}]
                        rWords1[oWord] = {rWord:meas}
    print("size of rWords1 =", len(rWords1))

In [31]:
#rWords1 = dict()
#import operator
#print(rWords1["consum"])
#print(sorted(rWords1["consum"], key=operator.itemgetter(1)))

In [32]:
for j, (ow, rws) in  enumerate(rWords.iteritems()):
    print (ow)
    for i, rw in enumerate(rws):
        print ("--> ", rw)
        if i>10:
            break
    if j>20:
        break


-->  {'pawn': 1.0}
-->  {'pawn': 1.0}
-->  {'rook': 1.0}
-->  {'rook': 1.0}
on reason person book ticket
-->  {'go on vacat': 1.0}
encount with friend
-->  {'plai basketbal': 1.0}
bean-stalk
-->  {'bean': 1.0}
antitrad wind
-->  {'antitrad': 1.0}
-->  {'prevail wind': 1.58496250072}
canada goos
-->  {'canada': 1.0}
-->  {'larg north american bird that honk': 1.58496250072}
spiderl
-->  {'-ling': 1.0}
vinalon
-->  {'vinyl': 1.0}
utnapishtim
-->  {'semit deiti': 1.58496250072}
spideri
-->  {'spider': 1.0}
calvin-benson cycl
-->  {'calvin cycl': 1.0}
salt flat
-->  {'flat': 1.58496250072}
steller sea-lion
-->  {'eumetopia jubatu': 1.0}
fuck your mom in ass
-->  {'regard as taboo': 1.0}
nunneri
-->  {'brothel': 1.0}
-->  {'convent': 1.58496250072}
-->  {'nun': 1.0}
ecolog servic
-->  {'servic': 1.0}
reel in
-->  {'reel': 1.0}
w:polish corridor
-->  {'corridor': 1.0}
cardiospermum
-->  {'dicot genu': 1.58496250072}
-->  {'sapindacea': 1.58496250072}
modelessli
-->  {'model': 1.0}
houyhnhnm

In [33]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords(rWords, rWords1, origWords, expansionCount):
#origWords = ["consum", "laura"]
    #text_string = ""
    qRWords01 = dict()
    for ow in origWords:
        #print(ow, rWords[ow])
        #print(ow)
        qRWords = set()
        #print("rWords =", rWords[ow])
        #rWords_ow = dict((i.keys()[0], sum(d.get(i.keys()[0], 0) for d in rWords[ow])) for i in rWords[ow])
        rWords_ow = dict((i.keys()[0], sum(d.get(i.keys()[0], 0) for d in rWords.get(ow, dict()))) for i in rWords.get(ow, dict()))
        #break
        if ow in rWords:
            qRWords = set([i.keys()[0] for i in rWords[ow]])
        
        #qRWords1 = rWords1[ow]
        #print("ow, qRWords =  ", ow, qRWords)

        #print("----------------\n")
        qRWords1 = rWords1.get(ow, dict())
        #print("ow, qRWords1 =  ", ow, qRWords1)
        #for qrw in qRWords:
        cnetCoeff = 0.2;
        for qrw, meas in rWords_ow.iteritems():
            if qrw in qRWords01:
                qRWords01[qrw] += (1-cnetCoeff) * qRWords1.get(qrw, 0) + cnetCoeff * meas
            else:
                #if qRWords1.get(qrw, 0) > 0:
                qRWords01[qrw] = (1-cnetCoeff) * qRWords1.get(qrw, 0) + cnetCoeff * meas
    sorted_qRWords01 = sorted(qRWords01.items(), key=operator.itemgetter(1), reverse=True)
    #print("sorted_qRWords01 =",sorted_qRWords01)
    text_string = ""
    for rw, v in sorted_qRWords01[0: expansionCount]:
        #print ("rw =", rw)
        if  all(c in string.printable for c in rw):
            rw = regex.sub(' ', rw)
            #rw = "".join([ c for c in rw if c.isalpha() or c == " " ])
            text_string += rw + " "
    #text_string = ' '.join([i[0] for i in sorted_qRWords01[0: expansionCount]])
#    text_string = ' '.join(sorted_qRWords01.keys()[0: expansionCount])
    return(text_string)
"""
    qRWords = qRWords | rWords_s
qRWords_d = dict()
for w in qRWords:
    rWords1[w][]
    if w in qRWords_d:
        qRWords_d[w] += rWords1[w]
    else:
        qRWords_d[w] = 
print(qRWords)
"""

'\n    qRWords = qRWords | rWords_s\nqRWords_d = dict()\nfor w in qRWords:\n    rWords1[w][]\n    if w in qRWords_d:\n        qRWords_d[w] += rWords1[w]\n    else:\n        qRWords_d[w] = \nprint(qRWords)\n'

In [74]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genQueries(intCoeff, expansionCount, dirCoeff):
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soup = BeautifulSoup(reader, 'lxml')
        soupNew = BeautifulSoup("", 'lxml')
        parameters_tag = soupNew.new_tag("parameters")
        index_tag = soupNew.new_tag("index")
        index_tag.string = colIndexDir
        parameters_tag.append(index_tag)
        if collection in {"aquaint", "gov"}:
            tags = soup.find_all(['doc'])
        elif collection in {"robust", "trec7n8"}:
            tags = soup.find_all(['top'])
            #print(tags)

        for tag in tags:
            #print("tag =", tag)
            #print('docno =', tag.find('docno').string)
            #print('text =', tag.find('text').string.strip())
            doc_tag = soupNew.new_tag("query")

            docno_tag = soupNew.new_tag("number")
            if collection in {"aquaint", "gov"}:
                docno_tag.string = (tag.find('docno').string).strip()
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<num> Number: (.*)\n', str(tag))                
                docno_tag.string = result.group(1).strip()
            doc_tag.append(docno_tag)

            text_tag = soupNew.new_tag("text")
            text_tag.string = "#weight(" 

            if collection in {"aquaint", "gov"}:
                origWords = splitStemText(tag.find('text').string.strip())
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<title>(.*)\n', str(tag))                
                origWords = splitStemText(result.group(1).strip())
            #print("origWords =", origWords)
                    
            text_tag.string += str(1-intCoeff) + " #combine(" 
            for ow in set(origWords):
                ow = regex.sub(' ', ow)
                text_tag.string += ow + " "
            text_tag.string += ") "

            text_string = ""
            #if len(rWords1) > 0:
            """
            text_string = getRelWords(rWords, rWords1, origWords, expansionCount)
           
            else:
            
            for ow in origWords:
                relWords = rWords.get(ow, [])[0: expansionCount]
                #relWords = getRelatedWords(ow) 
                #print("relWords =", relWords)
                for rw in {i.keys()[0] for i in relWords}:
                    if  all(c in string.printable for c in rw):
                        rw = regex.sub(' ', rw)
                        #rw = "".join([ c for c in rw if c.isalpha() or c == " " ])
                        text_string += rw + " "
            """
            for ow in origWords:
                #relWords1 = rWords1.get(ow, [])[0: expansionCount]
                #qrWords_ow = dict((i.keys()[0], sum(d.get(i.keys()[0], 0) for d in rWords[ow])) for i in rWords[ow])
                #qrWords_ow = dict((i.keys()[0], sum(d.get(i.keys()[0], 0) for d in rWords.get(ow, dict()))) for i in rWords.get(ow, dict()))
                relWords = rWords.get(ow, [])
                #print("relWords1 =", relWords1)
                #relWords = getRelatedWords(ow) 
                #print("relWords =", relWords)
                #qRWords = rWords.get(ow, dict())
                #print("qRWords =", qRWords)
                counter = 0
                #print("relWords =", relWords)
                #for rw in {i for i in relWords}:
                for rw in {i.keys()[0] for i in relWords}:
                    #print("--->", rw)
                    #if rw not in qrWords_ow.keys():
                    #    continue
                    if (counter>expansionCount):
                        break
                    if  all(c in string.printable for c in rw):
                        rw = regex.sub(' ', rw)
                        #rw = "".join([ c for c in rw if c.isalpha() or c == " " ])
                        text_string += rw + " "
                    counter += 1
                    
            #print("text_string =", text_string)
            if len(text_string)>3:
                text_tag.string += str(intCoeff) + " #combine(" 
                #print("text_string =", text_string)
                text_tag.string += text_string.encode('utf-8')
                text_tag.string += ") "

            text_tag.string += ") "

            doc_tag.append(text_tag)

            parameters_tag.append(doc_tag)
            #print(doc_tag)

        rule_tag = soupNew.new_tag("rule")
        rule_tag.string = "method:dir,mu:" + str(dirCoeff)
        #rule_tag.string = "method:two"
        parameters_tag.append(rule_tag)

        #intCoeff_tag = soupNew.new_tag("intCoeff")
        #intCoeff_tag.string = "0.8"
        #parameters_tag.append(intCoeff_tag)

        threads_tag = soupNew.new_tag("threads")
        threads_tag.string = "32"
        parameters_tag.append(threads_tag)

        count_tag = soupNew.new_tag("count")
        count_tag.string = "1000"
        parameters_tag.append(count_tag)

        trecFormat_tag = soupNew.new_tag("trecFormat")
        trecFormat_tag.string = "true"
        parameters_tag.append(trecFormat_tag)

        soupNew.append(parameters_tag)
        #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgOutFileName, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</text>", "\n</text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")
        outFile.write(soupNewStr)

In [75]:
def precisionCompute(intCoeffs, expansionCounts, dirCoeffs):
    mapPrecs = dict()
    for intCoeff in intCoeffs:
        for expansionCount in expansionCounts:
            for dirCoeff in dirCoeffs:
                genQueries(intCoeff, expansionCount, dirCoeff)
                subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                #print ("cmd = ", cmd)
                mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                print("intCoeff, expansionCount, dirCoeff, map precision =", intCoeff, expansionCount, dirCoeff, mapPrec)
                mapPrecs[intCoeff] = mapPrec
    return mapPrecs

In [ ]:
#intCoeffs = [0, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
#intCoeffs = [ 0.6 ]
#intCoeffs = [0.5]
#intCoeffs = np.arange(0.1, 0.9, 0.1)
intCoeffs = [ 0.3 ]
expansionCounts = [ 85 ]
dirCoeffs = [ 600 ]
dirCoeffs = range(200, 4000, 200)
mapPrecs = precisionCompute(intCoeffs, expansionCounts, dirCoeffs)
